# NONCOVToolbox: Step 2
## Generate the datasets from the ORCA output files with @OrcaAnalysis

Use the OrcaAnalysis module of the NONCOVToolbox to postprocess the ORCA output files from DFT calculations and save data to csv pandas dataframe.

### Load necessary modules from the NONCOVToolbox src

In [69]:
# Get the NONCOVToolbox library and print header
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
from sklearn.cluster import KMeans
import pathlib as Path
import importlib

path_noncov = os.path.abspath(os.path.join('..', '..', 'src'))

if path_noncov not in sys.path:
    sys.path.append(path_noncov)

from noncov import NONCOVToolbox, NONCOVHeader

noncov = NONCOVToolbox()

#NONCOVHeader.print_header()

# OrcaAnalysis module for postprocessing of DFT calculations
from noncov import OrcaAnalysis

# Functions to store data in dataframes
from noncov import MachineLearning

# Show performance and features of various NMR functions in module
from noncov import NMRFunctions

# Disable printing
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore printing
def enablePrint():
    sys.stdout = sys.__stdout__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
# Get work directory and scratch folder for the output data
current_dir = os.getcwd()
print(f'Current work directory is: {current_dir}')

scratch_dir = os.path.abspath(os.path.join('..', '..', 'scratch'))
print(f'Current scratch directory is: {scratch_dir}')
scratch_dir = OrcaAnalysis().convert_path(scratch_dir)

Current work directory is: /Users/ettorebartalucci/Desktop/NONCOV/examples/KLaL_Cation_Pi_SGP
Current scratch directory is: /Users/ettorebartalucci/Desktop/NONCOV/scratch
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch


### Preallocate two empty dataframes for ML applications

In [81]:
# Check whether a database exists in your directory and create it if it doesnt
datasets_dir = os.path.join(scratch_dir, 'GenerateMLDataset/data/KLaL2/')
print(f'Dataset directory is: {datasets_dir}')
datasets_dir = OrcaAnalysis().convert_path(datasets_dir)

dataset_name = 'klal_gua_sgp_nmr.csv'

if os.listdir(datasets_dir) == []:
    print("No files found in the directory, creating datasets... \n")
    
    # Make the dataset for the individual NMR properties
    MachineLearning().make_empty_nuc_prop_df(datasets_dir, dataset_name)
else:
    print("Some files found in the directory, skipping... \n")

Dataset directory is: /Users/ettorebartalucci/Desktop/NONCOV/scratch/GenerateMLDataset/data/KLaL2/
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/GenerateMLDataset/data/KLaL2/
No files found in the directory, creating datasets... 

The empty nuclear property dataset has been created and saved in: /Users/ettorebartalucci/Desktop/NONCOV/scratch/GenerateMLDataset/data/KLaL2/klal_gua_sgp_nmr.csv




In [99]:
# Display how the empty databases look like
nucprop = os.path.join(datasets_dir, dataset_name)
nucprop_df = pd.read_csv(nucprop)

In [10]:
# Individual nuclear properties
nucprop_df

,Molecule,Atom,x_coord,y_coord,z_coord,sigma_iso,sigma_xx,sigma_yy,sigma_zz,dia_sigma_xx,...,dia_sigma_zz,para_sigma_xx,para_sigma_yy,para_sigma_zz,sigma_11,sigma_22,sigma_33,s_tot_symmetry,span,skew


### Process ORCA calculations with @ORCAAnalysis

In [12]:
# Provide files you want to process as input 
orca_output = input("Enter the path to the ORCA file you want to work with: ")
orca_output = OrcaAnalysis().convert_path(orca_output)

Enter the path to the ORCA file you want to work with:  KLaL_cation_pi_RCCE_opt.mpi8.out


Normalized path using os.path: KLaL_cation_pi_RCCE_opt.mpi8.out


In [14]:
# Get the head of the file for saving files later
basename = os.path.basename(orca_output)
outname = basename.split('.')[0]

In [16]:
# Get a list of all the molecule names in the calculation, needed for ML later
list_molecules = OrcaAnalysis().extract_molecule_names(orca_output)
print(f'You have calculated the following molecules: {list_molecules}\n')

You have calculated the following molecules: ['KLaL_cation_pi_RCCE_opt_neg1p5.xyz', 'KLaL_cation_pi_RCCE_opt_neg1p4.xyz', 'KLaL_cation_pi_RCCE_opt_neg1p3.xyz', 'KLaL_cation_pi_RCCE_opt_neg1p2.xyz', 'KLaL_cation_pi_RCCE_opt_neg1p1.xyz', 'KLaL_cation_pi_RCCE_opt_neg1p0.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p9.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p8.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p7.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p6.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p5.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p4.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p3.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p2.xyz', 'KLaL_cation_pi_RCCE_opt_neg0p1.xyz', 'KLaL_cation_pi_RCCE_opt.xyz', 'KLaL_cation_pi_RCCE_opt_0p1.xyz', 'KLaL_cation_pi_RCCE_opt_0p2.xyz', 'KLaL_cation_pi_RCCE_opt_0p3.xyz', 'KLaL_cation_pi_RCCE_opt_0p4.xyz', 'KLaL_cation_pi_RCCE_opt_0p5.xyz', 'KLaL_cation_pi_RCCE_opt_0p6.xyz', 'KLaL_cation_pi_RCCE_opt_0p7.xyz', 'KLaL_cation_pi_RCCE_opt_0p8.xyz', 'KLaL_cation_pi_RCCE_opt_0p9.xyz', 'KLaL_cation_pi_RCCE_opt_1p0.xyz', 'KL

In [18]:
len(list_molecules)

56

In [71]:
# Working with ORCA .out files

# Count how many sequential calculations have been done
n_jobs = OrcaAnalysis().count_jobs_number(orca_output)
print(f'Number of ORCA jobs in file: {n_jobs}\n')

# Compute size of the .out file and suggest Git LFS 
size_orca_output = os.path.getsize(orca_output)
size_orca_output = size_orca_output/1e6
print(f'Size of ORCA file is: {size_orca_output} MB\n')

if n_jobs > 20:
    print(f'Careful, you are working with a possibly large output file of several GB\n')
    print(f'If using version controls consider setting up a .gitignore \n')

if size_orca_output > 1:
    print(f"Careful, you are working with a '{size_orca_output}' KB large file..\n")
    print(f'Set up a .gitignore or Git LFS before pushing to Git\n')

# Extract level of theory
lot_out = OrcaAnalysis().extract_level_of_theory(orca_output)
print(f'Level of theory for the NMR calculations is: {lot_out}\n')

# Split orca output in several subfiles for ease of handling (takes a while)
if n_jobs > 2:
    print('Your output file will be now spilt into subfiles. \n')
    #OrcaAnalysis().split_orca_output(scratch_dir, orca_output)

else:
    print("This is a one job file")

Number of ORCA jobs in file: 56

Size of ORCA file is: 8669.30638 MB

Careful, you are working with a possibly large output file of several GB

If using version controls consider setting up a .gitignore 

Careful, you are working with a '8669.30638' KB large file..

Set up a .gitignore or Git LFS before pushing to Git

Level of theory for the NMR calculations is: Job started from odin1, running /scratch/bartalucci/KLaL_cation_pi_RCCE_opt__VnjacZ__113301/orca/orca

Your output file will be now spilt into subfiles. 



In [89]:
# Initialize variables for shielding tensor components

S_dia = []
S_para = []
S_tot = []
nuclear_identities = []
mayer_bo = []
nuc_coords = []

# Extract NMR data from each splitted file
for job_number in range (1, n_jobs+1): # split files = number of jobs
        
    #blockPrint()
    
    # Path to the splitted outputs from the .out MPI8 file
    orca_splitted_output = OrcaAnalysis().convert_path(os.path.join(scratch_dir, 'OrcaAnalysis/split_orca_output', f'splitted_orca_job{job_number}.out'))

    # Extract CSA data
    shielding_dia, shielding_para, shielding_tot, nucleus_info = OrcaAnalysis().extract_tensor_data(orca_splitted_output)

    # Here include j coupling extraction
    #-------------
    
    # Extract bond orders
    #bond_orders = OrcaAnalysis().extract_mayer_bond_order(orca_splitted_output)
    
    # Print the bond orders and their interacting nuclei
    #for nucleus, bonds in bond_orders.items():
    #    print(f"{nucleus}:")
    #    for interacting_nucleus, bond_order in bonds:
    #       print(f"  Bond with {interacting_nucleus}: {bond_order}")
    
    #enablePrint()
    
    coords = OrcaAnalysis().extract_xyz_coords(orca_splitted_output)
    
    # Append shielding tensor matrices (non-diagonalized) - all nuclei for each job iteration
    S_dia.append(shielding_dia)
    S_para.append(shielding_para)
    S_tot.append(shielding_tot)
    nuclear_identities.append(nucleus_info)
    
    # Append bond orders
    #mayer_bo.append(bond_orders)
    
    # Append coordinates
    nuc_coords.append(coords)

Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job1.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job2.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job3.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job4.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job5.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job6.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratch/OrcaAnalysis/split_orca_output/splitted_orca_job7.out
Normalized path using os.path: /Users/ettorebartalucci/Desktop/NONCOV/scratc

In [91]:
S_dia

[{'Nucleus 3H :': [[-35.648, -28.977, -124.394],
   [15.621, 24.423, 40.959],
   [-17.604, 5.687, 21.365]],
  'Nucleus 4H :': [[-31.673, -25.637, -82.878],
   [11.235, 61.353, 91.874],
   [-9.317, 4.406, 15.81]],
  'Nucleus 7H :': [[-24.142, -9.919, -88.184],
   [6.922, 80.487, -36.714],
   [-13.383, -3.19, 45.034]],
  'Nucleus 9H :': [[57.3, -0.853, -58.172],
   [-4.996, 61.931, -19.44],
   [1.51, 0.085, 36.057]],
  'Nucleus 11H :': [[133.119, -1.012, -40.57],
   [-10.002, 34.695, -28.655],
   [8.892, 0.202, 52.952]],
  'Nucleus 14H :': [[92.797, -1.415, -31.774],
   [-15.07, 48.711, 47.104],
   [4.895, -7.323, 22.612]],
  'Nucleus 15H :': [[134.199, -0.513, -43.471],
   [-17.409, 57.829, 8.278],
   [17.157, -12.674, 37.782]],
  'Nucleus 18H :': [[115.605, 25.009, -18.72],
   [0.355, 78.227, -39.303],
   [7.569, -7.643, 48.124]],
  'Nucleus 20H :': [[78.566, 1.211, 6.948],
   [4.051, 93.546, -49.304],
   [-3.609, 1.025, 45.431]],
  'Nucleus 22H :': [[107.568, -22.067, 52.801],
   [9.3

In [103]:
# Get all the data that are not pairwise
data = []

# Loop through the number of jobs and get each molecule, each job has a different one
for job_number in range(0, n_jobs):
    molecule_name = list_molecules[job_number]
    
    # Process each job for S_tot
    shielding_dict = S_tot[job_number]
    if isinstance(shielding_dict, dict):
        for nucleus_index, (nucleus_key, tensor) in enumerate(shielding_dict.items()):
            shielding_tensor, s_iso, diagonal_mehring, eigenvals, eigenvecs, symmetry, span, skew = NMRFunctions().diagonalize_tensor(tensor)
            
            sigma_xx = eigenvals[0]
            sigma_yy = eigenvals[1]
            sigma_zz = eigenvals[2]
            
            sigma_11 = diagonal_mehring[0][0]
            sigma_22 = diagonal_mehring[1][1]
            sigma_33 = diagonal_mehring[2][2]
            
            # Extract coordinates and identities for the current nucleus
            nuc_id = nuclear_identities[job_number][nucleus_index]
            
            # Handle different structures of nuc_coords
            coords = nuc_coords[job_number][nucleus_index]
            if len(coords) >= 3:
                x_coord = coords[1]
                y_coord = coords[2]
                z_coord = coords[3]
            else:
                x_coord = y_coord = z_coord = None  

            # Collect the data for this nucleus
            row_data = {
                'Molecule': molecule_name,
                'Atom': nuc_id,
                'x_coord': x_coord,
                'y_coord': y_coord,
                'z_coord': z_coord,
                'sigma_iso': s_iso,
                'sigma_xx': sigma_xx,
                'sigma_yy': sigma_yy,
                'sigma_zz': sigma_zz,
                'dia_sigma_xx': None,  
                'dia_sigma_yy': None,
                'dia_sigma_zz': None,
                'para_sigma_xx': None,  
                'para_sigma_yy': None,
                'para_sigma_zz': None,
                'sigma_11' : sigma_11,
                'sigma_22' : sigma_22,
                'sigma_33' : sigma_33,
                's_tot_symmetry' : symmetry,
                'span' : span,
                'skew' : skew
            }
            data.append(row_data)

    # After collecting data from S_tot, update with data from S_dia
    shielding_dict = S_dia[job_number]
    if isinstance(shielding_dict, dict):
        for nucleus_index, (nucleus_key, tensor) in enumerate(shielding_dict.items()):
            dia_shielding_tensor, dia_s_iso, dia_diagonal_mehring, dia_eigenvals, dia_eigenvecs, dia_symmetry, dia_span, dia_skew = NMRFunctions().diagonalize_tensor(tensor)

            dia_sigma_xx = dia_eigenvals[0]
            dia_sigma_yy = dia_eigenvals[1]
            dia_sigma_zz = dia_eigenvals[2]

            # Update the existing row_data with dia_sigma values
            for row in data:
                if row['Molecule'] == molecule_name and row['Atom'] == nucleus_key:
                    row.update({
                        'dia_sigma_xx': dia_sigma_xx,
                        'dia_sigma_yy': dia_sigma_yy,
                        'dia_sigma_zz': dia_sigma_zz
                    })

    # After collecting data from S_dia, update with data from S_para
    shielding_dict = S_para[job_number]
    if isinstance(shielding_dict, dict):
        for nucleus_index, (nucleus_key, tensor) in enumerate(shielding_dict.items()):
            para_shielding_tensor, para_s_iso, para_diagonal_mehring, para_eigenvals, para_eigenvecs, para_symmetry, para_span, para_skew = NMRFunctions().diagonalize_tensor(tensor)
            
            para_sigma_xx = para_eigenvals[0]
            para_sigma_yy = para_eigenvals[1]
            para_sigma_zz = para_eigenvals[2]

            # Update the existing row_data with para_sigma values
            for row in data:
                if row['Molecule'] == molecule_name and row['Atom'] == nucleus_key:
                    row.update({
                        'para_sigma_xx': para_sigma_xx,
                        'para_sigma_yy': para_sigma_yy,
                        'para_sigma_zz': para_sigma_zz
                    })

# Convert the list of rows to a DataFrame
datadf = pd.DataFrame(data)

# Concatenate with the existing DataFrame
nucprop_df = pd.concat([nucprop_df, datadf], ignore_index=True)

# Save the updated DataFrame to a CSV file
nucprop_df.to_csv(nucprop, index=False)

# Display the updated DataFrame
nucprop_df


# -------------------------------------------------- #
# TENSOR DIAGONALIZATION FUNCTION HAS BEEN REQUESTED #


Shielding Tensor is: 
[[31.731  1.296 -5.381]
 [-0.843 23.606 -0.677]
 [-3.535 -1.344 27.096]]
Proceeding to transposing...

Transposed matrix is: 
[[31.731 -0.843 -3.535]
 [ 1.296 23.606 -1.344]
 [-5.381 -0.677 27.096]]
Proceeding to symmetrization...

Symmetric tensor is: 
[[31.731   0.2265 -4.458 ]
 [ 0.2265 23.606  -1.0105]
 [-4.458  -1.0105 27.096 ]]

Antisymmetric tensor is. 
[[ 0.      1.0695 -0.923 ]
 [-1.0695  0.      0.3335]
 [ 0.923  -0.3335  0.    ]]

Since antisymmetric part does not contribute to observable but only to relaxation, skipping...

Proceeding to diagonalization...

Eigenvalues are: [34.49 24.83 23.12], Eigenvectors are: 
[[ 0.85  0.48  0.22]
 [ 0.07 -0.51  0.86]
 [-0.52  0.72  0.46]]

Proceeding to ordering eigenvalues and eigenvectors...

Magnitude-based ordering of eigenvalues is: 
[23.12 24.83 34.49] 
 and of eigenvectors is: 
[[ 0.22  0.48  0.85]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/var/folders/s_/t420f2pn6xq2bv06qkv7bdp40000gn/T/ipykernel_77693/3221549994.py:102: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nucprop_df = pd.concat([nucprop_df, datadf], ignore_index=True)


,Molecule,Atom,x_coord,y_coord,z_coord,sigma_iso,sigma_xx,sigma_yy,sigma_zz,dia_sigma_xx,...,dia_sigma_zz,para_sigma_xx,para_sigma_yy,para_sigma_zz,sigma_11,sigma_22,sigma_33,s_tot_symmetry,span,skew
0,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 3H :,1.85056,-3.65399,-14.42354,27.48,34.49,24.83,23.12,-84.19,...,15.18,110.38,-49.03,10.94,23.12,24.83,34.49,0,11.37,-0.70
1,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 4H :,-1.00867,-3.16908,-13.28325,26.93,23.21,27.63,29.96,-63.17,...,99.67,88.27,19.12,-72.09,23.21,27.63,29.96,0,6.75,0.31
2,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 7H :,2.41198,-2.20461,-16.88088,24.21,29.69,20.16,22.79,-51.87,...,94.15,76.55,-34.07,-71.21,20.16,22.79,29.69,0,9.53,-0.45
3,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 9H :,3.34765,-1.9455,-17.38756,24.75,20.50,26.42,27.34,14.47,...,63.21,11.32,-51.80,-40.55,20.50,26.42,27.34,0,6.84,0.73
4,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 11H :,2.39691,-3.29532,-16.81784,24.16,29.66,20.16,22.66,136.21,...,59.12,-112.72,-0.88,-34.69,20.16,22.66,29.66,0,9.50,-0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 51C :,-0.64706,4.78323,-14.92389,28.37,-58.28,104.06,39.33,155.14,...,275.68,-83.31,-226.82,-335.90,-58.28,39.33,104.06,0,162.34,0.20
3468,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 57C :,-0.16135,2.99658,-15.68304,26.93,124.43,-25.19,-18.44,228.50,...,299.89,-111.28,-281.37,-321.04,-25.19,-18.44,124.43,0,149.62,-0.91
3469,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 52N :,-0.9608,5.43763,-13.83124,175.86,115.64,183.03,228.92,400.42,...,318.86,-240.69,-156.50,-129.94,115.64,183.03,228.92,0,113.28,0.19
3470,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 54N :,-0.9621,6.44135,-13.79599,176.95,229.48,181.91,119.47,197.29,...,360.00,-15.57,-193.70,-129.68,119.47,181.91,229.48,0,110.01,0.14


In [105]:
nucprop_df

,Molecule,Atom,x_coord,y_coord,z_coord,sigma_iso,sigma_xx,sigma_yy,sigma_zz,dia_sigma_xx,...,dia_sigma_zz,para_sigma_xx,para_sigma_yy,para_sigma_zz,sigma_11,sigma_22,sigma_33,s_tot_symmetry,span,skew
0,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 3H :,1.85056,-3.65399,-14.42354,27.48,34.49,24.83,23.12,-84.19,...,15.18,110.38,-49.03,10.94,23.12,24.83,34.49,0,11.37,-0.70
1,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 4H :,-1.00867,-3.16908,-13.28325,26.93,23.21,27.63,29.96,-63.17,...,99.67,88.27,19.12,-72.09,23.21,27.63,29.96,0,6.75,0.31
2,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 7H :,2.41198,-2.20461,-16.88088,24.21,29.69,20.16,22.79,-51.87,...,94.15,76.55,-34.07,-71.21,20.16,22.79,29.69,0,9.53,-0.45
3,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 9H :,3.34765,-1.9455,-17.38756,24.75,20.50,26.42,27.34,14.47,...,63.21,11.32,-51.80,-40.55,20.50,26.42,27.34,0,6.84,0.73
4,KLaL_cation_pi_RCCE_opt_neg1p5.xyz,Nucleus 11H :,2.39691,-3.29532,-16.81784,24.16,29.66,20.16,22.66,136.21,...,59.12,-112.72,-0.88,-34.69,20.16,22.66,29.66,0,9.50,-0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 51C :,-0.64706,4.78323,-14.92389,28.37,-58.28,104.06,39.33,155.14,...,275.68,-83.31,-226.82,-335.90,-58.28,39.33,104.06,0,162.34,0.20
3468,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 57C :,-0.16135,2.99658,-15.68304,26.93,124.43,-25.19,-18.44,228.50,...,299.89,-111.28,-281.37,-321.04,-25.19,-18.44,124.43,0,149.62,-0.91
3469,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 52N :,-0.9608,5.43763,-13.83124,175.86,115.64,183.03,228.92,400.42,...,318.86,-240.69,-156.50,-129.94,115.64,183.03,228.92,0,113.28,0.19
3470,KLaL_cation_pi_RCCE_opt_4p0.xyz,Nucleus 54N :,-0.9621,6.44135,-13.79599,176.95,229.48,181.91,119.47,197.29,...,360.00,-15.57,-193.70,-129.68,119.47,181.91,229.48,0,110.01,0.14
